<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/PFG_RNN_Predictiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Red Neuronal Predictiva

Creación de una red neuronal predictiva que sea capaz de clasificar nuevos usuarios con buena precisión en servicio y provincia.

In [103]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [104]:
pip install unidecode

In [105]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt
import unicodedata

In [ ]:
# Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
folder_path = '/content/drive/My Drive/PFG_FASPAS/RNN'
docs_xlsx = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
print(docs_xlsx)

In [ ]:
dic_dataframes = {}

for doc in docs_xlsx:
    entire_path = os.path.join(folder_path, doc)
    df = pd.read_excel(entire_path)
    # Uso el nombre del archivo como clave
    dic_dataframes[doc] = df

# Imprimir todas las claves
for clave in dic_dataframes.keys():
    print(clave)

In [ ]:
RN_SAAF = dic_dataframes.get("RN_SAAF.xlsx")
RN_AT = dic_dataframes.get("RN_AT.xlsx")
# RN_ACT = dic_dataframes.get("RN_ACT.xlsx")
RN_EM = dic_dataframes.get("RN_EM.xlsx")
RN_SO = dic_dataframes.get("RN_SO.xlsx")
RN_LOG = dic_zonas.get("RN_LOG.xlsx")
df_8000rows = dic_mun.get("df_8000_rows.xlsx")

In [106]:
# CÓDIGO SOLO PARA EL CURRO (PQ NO PUEDO ACCEDER AL DRIVE)
RN_SAAF = pd.read_excel('/content/RN_SAAF.xlsx')
RN_AT = pd.read_excel('/content/RN_AT.xlsx')
# RN_ACT = pd.read_excel('/content/RN_ACT.xlsx')
RN_EM = pd.read_excel('/content/RN_EM.xlsx')
RN_LOG = pd.read_excel('/content/RN_LOG.xlsx')
RN_SO = pd.read_excel('/content/RN_SO.xlsx')
df_8000rows = pd.read_excel('/content/df_8000_rows.xlsx')
provincia_municipio = pd.read_excel('/content/PROVINCIA_MUNICIPIO.xlsx')

In [107]:
# Convert all string columns to uppercase and remove accents/diacritics in combined_df
df_8000rows = df_8000rows.applymap(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)
provincia_municipio = provincia_municipio.applymap(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)

In [108]:
# Function to add PROVINCIA column based on LOCALIDAD
def add_provincia_column(df, provincia_mapping):
    df['PROVINCIA'] = df['LOCALIDAD'].map(provincia_mapping)
    return df

# Create a dictionary mapping from LOCALIDAD to PROVINCIA
localidad_to_provincia = provincia_municipio.set_index('MUNICIPIO')['PROVINCIA'].to_dict()

# List of RN dataframes to process
rn_dataframes = {'AT': RN_AT, 'EM': RN_EM, 'LOG': RN_LOG, 'SAAF': RN_SAAF, 'SO': RN_SO}

# Add the PROVINCIA column to each RN dataframe
rn_dfs_with_provincia = {name: add_provincia_column(df, localidad_to_provincia) for name, df in rn_dataframes.items()}

# Drop rows with null values in the 'PROVINCIA' column in each RN dataframe
rn_dfs_cleaned = {name: df.dropna(subset=['PROVINCIA']) for name, df in rn_dfs_with_provincia.items()}

In [109]:
# List of columns to keep
required_columns = ['LOCALIDAD', 'GENERO', 'YEAR NACIMIENTO', 'PROVINCIA']

# Function to ensure the dataframe has all required columns
def ensure_columns(df, columns):
    for column in columns:
        if column not in df.columns:
            df[column] = np.nan  # Create column with NaN values if it does not exist
    return df[columns]  # Return dataframe with only the required columns

# Ensure all dataframes have the required columns
rn_dfs_final = {name: ensure_columns(df, required_columns) for name, df in rn_dfs_cleaned.items()}

<ipython-input-109-605fc2068a84>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = np.nan  # Create column with NaN values if it does not exist


In [110]:
# Mapping dictionary for normalizing gender values
gender_mapping = {
    'FEMENINO': 'MUJER',
    'M': 'MUJER',
    'MASCULINO': 'HOMBRE',
    'H': 'HOMBRE'
}

# Function to normalize the 'GENERO' column in a dataframe
def normalize_gender_column(df):
    df['GENERO'] = df['GENERO'].map(lambda x: gender_mapping.get(x, x))
    return df

# Apply the normalization to the 'SO' dataframe
rn_dfs_final['SO'] = normalize_gender_column(rn_dfs_final['SO'])

<ipython-input-110-333b05a14062>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GENERO'] = df['GENERO'].map(lambda x: gender_mapping.get(x, x))


In [111]:
# Function to insert random registers into a dataframe where the 'GENERO' column is empty
def insert_random_registers(df, gender_distribution):
    # Find the indices where 'GENERO' is empty
    empty_indices = df[df['GENERO'].isna()].index

    # Generate random genders based on the given distribution for the empty indices
    genders = np.random.choice(
        ['MUJER', 'HOMBRE'],
        size=len(empty_indices),
        p=[gender_distribution['MUJER'], gender_distribution['HOMBRE']]
    )

    # Assign these random genders to the empty 'GENERO' rows
    df.loc[empty_indices, 'GENERO'] = genders
    return df

# Example of usage
gender_distribution = {'MUJER': 0.44, 'HOMBRE': 0.56}

# Insert random registers into a specific dataframe, e.g., 'ACT', where 'GENERO' is empty
rn_dfs_final['SAAF'] = insert_random_registers(rn_dfs_final['SAAF'], gender_distribution)
rn_dfs_final['SO'] = insert_random_registers(rn_dfs_final['SO'], gender_distribution)

In [112]:
# Get the min and max year in the 'YEAR NACIMIENTO' column
min_year = int(rn_dfs_final['SO']['YEAR NACIMIENTO'].min())
max_year = int(rn_dfs_final['SO']['YEAR NACIMIENTO'].max())

# Function to generate random years with the specified distribution
def generate_random_years(size):
    # 75% of the years under 2004
    under_2004 = np.random.randint(min_year, 2004, size=int(size * 0.75))
    # 25% of the years between 2004 and max_year
    over_2004 = np.random.randint(2004, max_year + 1, size=size - int(size * 0.75))
    return np.concatenate([under_2004, over_2004])

# Function to insert random years into the 'YEAR NACIMIENTO' column where it is empty
def insert_random_years(df):
    # Find the indices where 'YEAR NACIMIENTO' is empty
    empty_indices = df[df['YEAR NACIMIENTO'].isna()].index

    # Generate random years based on the specified distribution
    random_years = generate_random_years(len(empty_indices))
    np.random.shuffle(random_years)  # Shuffle to mix the values

    # Assign these random years to the empty 'YEAR NACIMIENTO' rows
    df.loc[empty_indices, 'YEAR NACIMIENTO'] = random_years
    return df

# Apply the function to the 'SO' dataframe
rn_dfs_final['SO'] = insert_random_years(rn_dfs_final['SO'])


In [113]:
# Add a column named 'SERVICIO' with the name of the dataframe to each dataframe in rn_dfs_final
def add_servicio_column(dfs):
    for name, df in dfs.items():
        df['SERVICIO'] = name
    return dfs

# Apply the function to rn_dfs_final
rn_dfs_final = add_servicio_column(rn_dfs_final)

<ipython-input-113-c5260d67a514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SERVICIO'] = name
<ipython-input-113-c5260d67a514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SERVICIO'] = name
<ipython-input-113-c5260d67a514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [119]:
# Combine all dataframes into a single dataframe
combined_df = pd.concat(rn_dfs_final.values(), ignore_index=True)

# Shuffle the rows of the combined dataframe
shuffled_combined_df = combined_df.sample(frac=1).reset_index(drop=True)

shuffled_combined_df['SERVICIO'].value_counts()

SERVICIO
SO      1457
SAAF     567
EM       203
LOG      182
AT       143
Name: count, dtype: int64

In [124]:
# Calculate the frequency of each element in the 'SERVICIO' column
service_frequency = df['SERVICIO'].value_counts()

# Group by 'PROVINCIA' and 'SERVICIO' and count the occurrences
service_province_freq = shuffled_combined_df.groupby(['PROVINCIA', 'SERVICIO']).size().reset_index(name='COUNT')

# Function to assign services based on frequency
def assign_service(row, freq_df):
    province = row['PROVINCIA']
    possible_services = freq_df[freq_df['PROVINCIA'] == province]
    if possible_services.empty:
        return np.nan
    else:
        return np.random.choice(possible_services['SERVICIO'].values, p=possible_services['COUNT'].values / possible_services['COUNT'].sum())

# Apply the function to assign services
df_8000rows['SERVICIO'] = df_8000rows.apply(assign_service, axis=1, args=(service_province_freq,))

# Ensure balanced classes by sampling
min_class_size = service_frequency.min()
balanced_df = df_8000rows.groupby('SERVICIO').apply(lambda x: x.sample(n=min_class_size, replace=True)).reset_index(drop=True)

In [127]:
balanced_df['SERVICIO'].shape

(7285,)